# Exemplo 04

## Tendo como base o exemplo 3, construímos um script mais dinâmico para:<br>  __autenticar__, __ler__, __atualizar__ e __criar__ nova planilha.<br> É necessário ter uma credencial de serviço.

<br> _Se você não executou o exemplo 03, faça a instalação da lib abaixo_
```
 pip install gspread
``` 

## __Bibliotecas__

[__Pandas__](https://pandas.pydata.org/pandas-docs/stable/index.html): Lib para nós ajudar com a manipulação dos dados. <br>
[__Gspread__](https://gspread.readthedocs.io/en/latest/index.html): Lib com as funções do GoogleSheets. <br>
[__Google auth__](https://google-auth.readthedocs.io/en/latest/reference/google.oauth2.service_account.html): Lib para autenticação.

In [2]:
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials

## __Função para autenticar no projeto do Google Developer__

Link de referência: https://gspread.readthedocs.io/en/latest/oauth2.html#for-bots-using-service-account

In [16]:
def cria_credenciais():
    scopes = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive'
        ]
        
    credentials = Credentials.from_service_account_file(
            'py013-spread-sheets-5d5449661c4e.json',
            scopes=scopes
        )
    
    return gspread.authorize(credentials)

## __Função para ler uma planilha__
Links de referência: https://gspread.readthedocs.io/en/latest/user-guide.html?#getting-all-values-from-a-row-or-a-column <br>
https://gspread.readthedocs.io/en/latest/user-guide.html?#getting-a-cell-value

In [17]:
def le_sheets(sample_spreadsheet_id, tab_index):
    gc = cria_credenciais()
    
    gc = gc.open_by_key(sample_spreadsheet_id)
    
    values = gc.get_worksheet(tab_index).get_all_values()
    
    df = pd.DataFrame(values)
    df.columns = df.iloc[0]
    df.drop(df.index[0], inplace=True)         

    return df

In [20]:
df_notas = le_sheets("1GYyp6cLDAhUDFz-itEUmp5lVxpqBJrj0075fjVVk7yM", 0 )

## __Função para atualizar uma planilha__
Link de referência: https://gspread.readthedocs.io/en/latest/user-guide.html?#updating-cells

In [19]:
def atualiza_sheets(id_aba, id_planilha, df):
    gc = cria_credenciais()
    
    gc = gc.open_by_key(id_planilha)
    
    sheet = gc.get_worksheet(id_aba)
    
    sheet.update(([df.columns.values.tolist()] + df.values.tolist()))

In [21]:
atualiza_sheets(2, "1GYyp6cLDAhUDFz-itEUmp5lVxpqBJrj0075fjVVk7yM", df_notas )

## __Função para criar uma nova planilha e atualizar__
Link de referência: https://gspread.readthedocs.io/en/latest/user-guide.html?#creating-a-worksheet

In [24]:
def cria_insere_sheets(nm_sheet, email_share, tipo_role, df):
    gc = cria_credenciais()
    
    sh = gc.create(nm_sheet)
    sh.share(email_share, perm_type='user', role=tipo_role)
    
    ## 
    
    atualiza_sheets(0, sh.id, df)
    return sh

In [25]:
cria_insere_sheets("Teste 01 Py013", "lmbr.45@gmail.com", "owner", df_notas)

<Spreadsheet 'Teste 01 Py013' id:1sa8K-sejwrsylA03wPiE8kDsILAw96FYdgx5Jdp-f9E>